In [ ]:
import sagemaker
from sagemaker import get_execution_role
from sagemaker.tuner import HyperparameterTuner
from sagemaker.sklearn.estimator import SKLearn
# S3 prefix

hpo_training_job = 'ENTER_HPO_JOBNAME_HERE'

assert(hpo_training_job != 'ENTER_HPO_JOBNAME_HERE')

sagemaker_session = sagemaker.Session()

tuner = HyperparameterTuner.attach(hpo_training_job, sagemaker_session=sagemaker_session)

tuner.wait()

In [ ]:
import pandas as pd

tuner_analysis = sagemaker.HyperparameterTuningJobAnalytics(tuner.latest_tuning_job.name)
full_df = tuner_analysis.dataframe()

full_df[['FinalObjectiveValue', 'batch_size', 'num_examples_per_epoch', 'TrainingJobName']]

In [ ]:
from sagemaker.tensorflow.estimator import TensorFlow
tensorflow_estimator = TensorFlow.attach('sagemaker-tensorflow-190116-1509-003-5811112b', sagemaker_session)
model = tensorflow_estimator.create_model()

In [ ]:
models = []
for index, row in full_df.iterrows():
    tensorflow_estimator = TensorFlow.attach(row['TrainingJobName'], sagemaker_session)
    model = tensorflow_estimator.create_model()
    model._create_sagemaker_model(instance_type='ml.m4.xlarge')
    models.append(model)

In [ ]:
productionVariants = []

for model_job in models:
    m = {}
    m['VariantName'] = model_job.name
    m['ModelName'] = model_job.name
    m['InitialInstanceCount'] = 1
    m['InstanceType'] = 'ml.m4.2xlarge'
    productionVariants.append(m)

    
endpointname = hpo_training_job + "-endpoint"
import boto3
client = boto3.client('sagemaker')
endpoint_config_resp = client.create_endpoint_config(EndpointConfigName=endpointname,
                                                    ProductionVariants= productionVariants)

print(endpoint_config_resp)

response = client.create_endpoint(
    EndpointName=endpointname,
    EndpointConfigName=endpointname
)

print(response)